In [63]:
import pandas as pd
import sys
import datetime
import numpy as np 
import scipy as sc
import scipy.stats

In [64]:
c1 = pd.concat(map(pd.read_csv, ['E0.csv']),ignore_index=True,sort = True)
c2 = pd.concat(map(pd.read_csv, ['E1.csv']),ignore_index=True,sort = True)
c3 = pd.concat(map(pd.read_csv, ['E2.csv']),ignore_index=True,sort = True)
c4 = pd.concat(map(pd.read_csv, ['E3.csv']),ignore_index=True,sort = True)
c5 = pd.concat(map(pd.read_csv, ['E4.csv']),ignore_index=True,sort = True)
c6 = pd.concat(map(pd.read_csv, ['E5.csv']),ignore_index=True,sort = True)

In [65]:
def add_game_week(df):
    MatchWeek = []
    for i, row in df.iterrows():
         df.loc[i,'MW']  = int((i+10) / 10)    
    return df

c1 = add_game_week(c1)
c2 = add_game_week(c2)
c3 = add_game_week(c3)
c4 = add_game_week(c4)
c5 = add_game_week(c5)
c6 = add_game_week(c6)

In [66]:
c1 = c1.iloc[::-1]
c1 = c1.reset_index(drop=True)
c2 = c2.iloc[::-1]
c2 = c2.reset_index(drop=True)
c3 = c3.iloc[::-1]
c3 = c3.reset_index(drop=True)
c4 = c4.iloc[::-1]
c4 = c4.reset_index(drop=True)
c5 = c5.iloc[::-1]
c5 = c5.reset_index(drop=True)
c6 = c6.iloc[::-1]
c6 = c6.reset_index(drop=True)

In [67]:
def calc_diff_between_odds(df):
    for i, row in df.iterrows():
        df.loc[i,'HOD'] = df.loc[i,'PSCH'] - df.loc[i,'B365H']
        df.loc[i,'DOD'] = df.loc[i,'PSCD'] - df.loc[i,'B365D']
        df.loc[i,'AOD'] = df.loc[i,'PSCA'] - df.loc[i,'B365A']    
    return df
        
c1 = calc_diff_between_odds(c1)
c2 = calc_diff_between_odds(c2)
c3 = calc_diff_between_odds(c3)
c4 = calc_diff_between_odds(c4)
c5 = calc_diff_between_odds(c5)
c6 = calc_diff_between_odds(c6)


c1 = c1[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW','HOD','DOD','AOD']]
c2 = c2[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW','HOD','DOD','AOD']]
c3 = c3[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW','HOD','DOD','AOD']]
c4 = c4[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW','HOD','DOD','AOD']]
c5 = c5[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW','HOD','DOD','AOD']]
c6 = c6[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW','HOD','DOD','AOD']]

In [68]:
c1 = c1[c1.HomeTeam.notnull()]
print(c1.isnull().values.any())

c2 = c2[c2.HomeTeam.notnull()]
print(c2.isnull().values.any())

c3 = c3[c3.HomeTeam.notnull()]
print(c3.isnull().values.any())

c4 = c4[c4.HomeTeam.notnull()]
print(c4.isnull().values.any())

c5 = c5[c5.HomeTeam.notnull()]
print(c5.isnull().values.any())

c6 = c6[c6.HomeTeam.notnull()]
print(c6.isnull().values.any())

False
False
False
False
False
False


In [69]:
def count_goal_diff(number_of_last_matches,team_side,column,df,counting_matches_from,column_score = None, column_lost = None):
    for i, row in df.iterrows():
        team = df.loc[i,team_side]
        goal_diff = 0
        goal_score = 0
        goal_lost =0
        iterator = 0
    
        for j in range(counting_matches_from,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == team:
                goal_diff += df.loc[(i+j),'FTHG'] - df.loc[(i+j),'FTAG']
                goal_score += df.loc[(i+j),'FTHG']
                goal_lost += df.loc[(i+j),'FTAG']
                iterator += 1
            elif df.loc[(i+j),'AwayTeam'] == team:
                goal_diff += df.loc[(i+j),'FTAG'] - df.loc[(i+j),'FTHG']
                goal_score += df.loc[(i+j),'FTAG']
                goal_lost += df.loc[(i+j),'FTHG']
                iterator += 1
            if iterator == number_of_last_matches:
                break
        df.loc[i,column] = goal_diff
        if column_score != None:
            df.loc[i,column_score] = goal_score 
        if column_lost != None:
            df.loc[i,column_lost] = goal_lost 
    
def count_goals_diff_for_both_teams(df):
    number_of_last_matches = 50
    count_goal_diff(number_of_last_matches,'HomeTeam','HGD',df,1,'HGS','HGL')
    count_goal_diff(number_of_last_matches,'AwayTeam','AGD',df,1,'AGS','AGL')
    
    count_goal_diff(50,'HomeTeam','HGDT',df,0)
    count_goal_diff(50,'AwayTeam','AGDT',df,0)
    
    for i, row in df.iterrows():
        df.loc[i,'GD'] = df.loc[i,'HGD'] - df.loc[i,'AGD']        
    return df

c1= count_goals_diff_for_both_teams(c1)
c2= count_goals_diff_for_both_teams(c2)
c3= count_goals_diff_for_both_teams(c3)
c4= count_goals_diff_for_both_teams(c4)
c5= count_goals_diff_for_both_teams(c5)
c6= count_goals_diff_for_both_teams(c6)

In [70]:
def get_points(result):
    if result == 'H':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
def get_points_reverse(result):
    if result == 'A':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

def calc_team_point(number_of_last_matches,team,i,df,counting_matches_from):
    points = 0
    iterator = 0
    for j in range(counting_matches_from,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == team:
                points += get_points(df.loc[(i+j),'FTR'])
                iterator += 1
            elif df.loc[(i+j),'AwayTeam'] == team:
                points += get_points_reverse(df.loc[(i+j),'FTR'])
                iterator += 1
            if iterator == number_of_last_matches:
                break 
    return points

def calc_points(df):
    number_of_last_matches = 50
    for i, row in df.iterrows():
        home_team = df.loc[(i),'HomeTeam']
        away_team = df.loc[(i),'AwayTeam']
        df.loc[i,'HP'] = calc_team_point(number_of_last_matches,home_team,i,df,1) 
        df.loc[i,'AP'] = calc_team_point(number_of_last_matches,away_team,i,df,1)
        df.loc[i,'HPT'] = calc_team_point(number_of_last_matches,home_team,i,df,0) 
        df.loc[i,'APT'] = calc_team_point(number_of_last_matches,away_team,i,df,0)
    
    for i, row in df.iterrows():
        df.loc[i,'DP'] = df.loc[i,'HP'] - df.loc[i,'AP']   
    return df

c1= calc_points(c1)
c2= calc_points(c2)
c3= calc_points(c3)
c4= calc_points(c4)
c5= calc_points(c5)
c6= calc_points(c6)

In [71]:
def get_points(result):
    if result == 'H':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
def get_points_reverse(result):
    if result == 'A':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
def get_match_result(points):
    if points == 3:
        return 'W'
    elif points == 1:
        return 'D'
    elif points == 0:
        return 'L'
    else:
        return 'N'

def calc_team_point(number_of_last_matches,team,i,df,counting_matches_from):
    points = ""
    iterator = 0
    for j in range(counting_matches_from,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == team:
                points += get_match_result(get_points(df.loc[(i+j),'FTR']))
                iterator += 1
            elif df.loc[(i+j),'AwayTeam'] == team:
                points += get_match_result(get_points_reverse(df.loc[(i+j),'FTR']))
                iterator += 1
            if iterator == number_of_last_matches:
                break 
    return points

def calc_points(df):
    number_of_last_matches = 5
    for i, row in df.iterrows():
        home_team = df.loc[(i),'HomeTeam']
        away_team = df.loc[(i),'AwayTeam']
        df.loc[i,'HF'] = calc_team_point(number_of_last_matches,home_team,i,df,1) 
        df.loc[i,'AF'] = calc_team_point(number_of_last_matches,away_team,i,df,1)
    
    return df

c1 = calc_points(c1)
c2 = calc_points(c2)
c3 = calc_points(c3)
c4 = calc_points(c4)
c5 = calc_points(c5)
c6 = calc_points(c6)

In [72]:
def calc_points_based_on_form(string):
    if string == 'WWWWW':
        return 5
    elif string[-4:] == 'WWWWW':
        return 4
    elif string[-3:] == 'WWW':
        return 3
    elif string[-2:] == 'WW':
        return 2
    elif string == 'LLLLL':
        return -5
    elif string[-4:] == 'LLLL':
        return -4
    elif string[-3:] == 'LLL':
        return -3
    elif string[-2:] == 'LL':
        return -2
    else:
        return 0
    
def check_last_teams_form(df):
    for i, row in df.iterrows():
        home_team_form = df.loc[(i),'HF']
        away_team_form = df.loc[(i),'AF']
        df.loc[i,'HLF'] = calc_points_based_on_form(home_team_form) 
        df.loc[i,'ALF'] = calc_points_based_on_form(away_team_form)
    
    for i, row in df.iterrows():
        df.loc[i,'LFD'] = df.loc[i,'HLF'] - df.loc[i,'ALF']
    
    return df

c1 = check_last_teams_form(c1)
c2 = check_last_teams_form(c2)
c3 = check_last_teams_form(c3)
c4 = check_last_teams_form(c4)
c5 = check_last_teams_form(c5)
c6 = check_last_teams_form(c6)

In [73]:
def get_team(df):
    return pd.DataFrame({'AwayTeam':df.AwayTeam.unique()})

def calc_points_table(df):
    dictionary = {}
    teams = get_team(df)
    for i, row in teams.iterrows():
        points = 0
        goal_diff = 0 
        team = teams.loc[(i),'AwayTeam']
        
        team_matches = df.loc[((df['AwayTeam'] == team) | (df['HomeTeam'] == team))]
        
        team_matches = team_matches.reset_index(drop=True)
        
        if team_matches.loc[(0),'HomeTeam'] == team:
            points = team_matches.loc[(0),'HPT']
            goal_diff = team_matches.loc[(0),'HGDT']
        elif team_matches.loc[(0),'AwayTeam'] == team:
            points = team_matches.loc[(0),'APT']
            goal_diff = team_matches.loc[(0),'AGDT']
            
        dictionary[team] = (points,goal_diff)
        
    return {key: rank for rank, key in enumerate(sorted(dictionary, key=dictionary.get, reverse=True), 1)}

def assing_position_in_table_to_team_in_last_season(table,df):
    positions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] 
    df['LSTPH'] = df['HomeTeam']
    df['LSTPA'] = df['AwayTeam']
    
    for team in get_team(df)['AwayTeam']:
        if not team in table:
            df.LSTPH = df.LSTPH.replace([team], [20])
            df.LSTPA = df.LSTPA.replace([team], [20])
            
    df.LSTPH = df.LSTPH.astype(str)
    df.LSTPA = df.LSTPA.astype(str)
    
    for key, value in table.items():
        df.LSTPH = df.LSTPH.replace([key], [value])
        df.LSTPA = df.LSTPA.replace([key], [value])
        
    df.LSTPH = df.LSTPH.astype(int)
    df.LSTPA = df.LSTPA.astype(int)
        
    for i, row in df.iterrows():
        df.loc[i,'LSTPD'] = df.loc[i,'LSTPH'] - df.loc[i,'LSTPA']
    
    return df

table_c2 = calc_points_table(c2)
table_c3 = calc_points_table(c3)
table_c4 = calc_points_table(c4)
table_c5 = calc_points_table(c5)
table_c6 = calc_points_table(c6)

c1 = assing_position_in_table_to_team_in_last_season(table_c2,c1)
c2 = assing_position_in_table_to_team_in_last_season(table_c3,c2)
c3 = assing_position_in_table_to_team_in_last_season(table_c4,c3)
c4 = assing_position_in_table_to_team_in_last_season(table_c5,c4)
c5 = assing_position_in_table_to_team_in_last_season(table_c6,c5)

In [74]:
def parse_day(df):
    newest_date = datetime.datetime.strptime(str(max(df['Date'])), '%Y-%m-%d').date()
    for i, row in df.iterrows():
        df.loc[i,'Day']  = df.at[i,'Date'].day 
        df.loc[i,'Month']  = df.at[i,'Date'].month
        df.loc[i,'Year']  = df.at[i,'Date'].year
        delta = newest_date - df.at[i,'Date']
        df.loc[i,'DaysFromToday']  = delta.days
        
def convert_float_to_int(df):
    columns_to_convert = ['FTAG','FTHG','Day','Month','Year','DaysFromToday']
    for c in columns_to_convert:
        df[c]= df[c].astype(int)
        
def convert_date(df):
    df['Date'] =pd.to_datetime(df['Date'],format='%d/%m/%y')
    df['Date'] = df['Date'].dt.date.apply(lambda x: datetime.date(x.year,x.month,x.day))
    df= df.sort_values(by=['Date'],ascending=False)
    parse_day(df)
    convert_float_to_int(df) 
    
    return df


c1= convert_date(c1)
c2= convert_date(c2)
c3= convert_date(c3)
c4= convert_date(c4)
c5= convert_date(c5)
c6= convert_date(c6)

In [75]:
filepaths = [c1,c2,c3,c4,c5]
df = pd.concat(filepaths,ignore_index=True,sort = True)

In [76]:
def get_points_between(result):
    if result == 'H':
        return 3
    elif result == 'D':
        return 0
    else:
        return -3
    
def get_points_reverse_between(result):
    if result == 'A':
        return 3
    elif result == 'D':
        return 0
    else:
        return -3

def calc_point(number_of_last_matches,home_team,away_team,i,df):
    points=0
    iterator = 0
    for j in range(1,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == home_team and df.loc[(i+j),'AwayTeam'] == away_team:
                points += get_points_between(df.loc[(i+j),'FTR'])
                iterator += 1
            elif df.loc[(i+j),'HomeTeam'] == away_team and df.loc[(i+j),'AwayTeam'] == home_team:
                points += get_points_reverse_between(df.loc[(i+j),'FTR'])
                iterator += 1
            if iterator == number_of_last_matches:
                break 
  
    df.loc[i,'DPB'] = points         
    return df

def calc_points_between_opponents(df):
    number_of_last_matches = 10
    for i, row in df.iterrows():
        home_team = df.loc[(i),'HomeTeam']
        away_team = df.loc[(i),'AwayTeam']
        df = calc_point(number_of_last_matches,home_team,away_team,i,df) 
    
    return df


df = calc_points_between_opponents(df)

In [77]:
columns = ['FTHG','FTAG','AGDT','APT','HGDT','HPT','Date']
df.drop(columns, inplace=True, axis=1)

In [78]:
df.to_csv('football_data.csv', sep='\t')